Conexão do Psycopg com o Banco de dados

In [9]:
import psycopg2

conn = psycopg2.connect(host="200.129.44.249", database="553095", user="553095", password="553095")

cur = conn.cursor()


Criação Tabelas

In [10]:
cur.execute("DROP CASCADE TABLE IF EXISTS public.Curso")
cur.execute("CREATE TABLE public.Curso(id INT, nome VARCHAR(100), regime VARCHAR(100), duracao INT, primary key(id))")

cur.execute("DROP TABLE IF EXISTS public.Aluno")
cur.execute("CREATE TABLE public.aluno(id INT, nome VARCHAR(100), curso_id INT, semestre INT, PRIMARY KEY(id), FOREIGN KEY(curso_id) REFERENCES Curso(id))")

cur.execute("DROP TABLE IF EXISTS public.Professor")
cur.execute("CREATE TABLE public.Professor(id INT, nome VARCHAR(100), area_especializacao VARCHAR(100), contato VARCHAR(100), curso_id INT, PRIMARY KEY(id), FOREIGN KEY(curso_id) REFERENCES Curso(id))")

cur.execute("DROP TABLE IF EXISTS public.Disciplina")
cur.execute("CREATE TABLE public.Disciplina(id INT, codigo VARCHAR(10) UNIQUE, nome VARCHAR(100), area_especializacao VARCHAR(100), curso_id INT, PRIMARY KEY(id), FOREIGN KEY(curso_id) REFERENCES Curso(id))")

cur.execute("DROP TABLE IF EXISTS public.Turma")
cur.execute("CREATE TABLE public.Turma(id INT, codigo VARCHAR(10) UNIQUE, disciplina_id INT, semestre VARCHAR(20), capacidaded_maxima INT, estado VARCHAR(20), prof_id INT, PRIMARY KEY(id), FOREIGN KEY(disciplina_id) REFERENCES Disciplina(id), FOREIGN KEY(prof_id) REFERENCES Professor(id))")

cur.execute("DROP TABLE IF EXISTS public.Aluno_Turma")
cur.execute("CREATE TABLE public.Aluno_Turma(aluno_id INT, turma_id INT, PRIMARY KEY(aluno_id, turma_id), FOREIGN KEY(aluno_id) REFERENCES Aluno(id), FOREIGN KEY(turma_id) REFERENCES Turma(id))")



conn.commit()


DependentObjectsStillExist: cannot drop table curso because other objects depend on it
DETAIL:  constraint aluno_curso_id_fkey on table aluno depends on table curso
HINT:  Use DROP ... CASCADE to drop the dependent objects too.
